In [ ]:
import CompDoobTransform as cdt
import time
import math
import torch
import matplotlib.pyplot as plt
from CompDoobTransform.utils import normal_logpdf, resampling
plt.style.use('ggplot')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Computing on ' + str(device))

In [ ]:
# dict for objects relating to latent state process
state = {}

# dimension of state 
d = 1 
state['dim'] = d

# drift of diffusion
beta = torch.tensor(1.0, device = device) 
b = lambda x: - beta * x # drift
state['drift'] = b

# diffusion coefficient of diffusion
sigma = torch.tensor(1.0, device = device) # diffusion coefficient
state['sigma'] = sigma

# simulate initial states (from stationary distribution)
initial = lambda N: (sigma / torch.sqrt(2.0 * beta)) * torch.randn(N, d, device = device)
state['initial'] = initial

# time interval
T = torch.tensor(1.0, device = device) 
state['terminal_time'] = T

In [ ]:
# dict for objects relating to observations
obs = {}

# dimension of observation
p = 1
obs['dim'] = p

# observation parameters
std_obs = 0.25
var_obs = torch.tensor(std_obs**2, device = device) # variance of observation

# log-observation density
obs_log_density = lambda x, y: normal_logpdf(y, x, var_obs) # terminal condition, returns size (N)
obs['log_density'] = obs_log_density

# simulate observations
observation = lambda N: initial(N) + torch.sqrt(var_obs) * torch.randn(N, p, device = device)
obs['observation'] = observation

In [ ]:
# define standardization means and standard deviations
standardization = {'x_mean': torch.zeros(1), 
                   'x_std': (sigma / torch.sqrt(2.0 * beta)) * torch.ones(1), 
                   'y_mean': torch.zeros(1), 
                   'y_std': torch.sqrt(sigma**2 / (2.0 * beta) + var_obs) * torch.ones(1)}

standardization

In [ ]:
# transition mean and variance 
mean_x = lambda t,x: x * torch.exp(- beta * t)
var_x = lambda t: (1.0 - torch.exp(- 2.0 * beta * t)) / (2 * beta)

# posterior mean and variance at time T
var_h = lambda t: 1.0 / (1.0 / var_x(t) + 1.0 / var_obs)
mean_h = lambda x0,y: var_h(T) * (mean_x(T, x0) / var_x(T) + y / var_obs)

# h function
log_h = lambda t,x,y: - 0.5 * d * torch.log(torch.tensor(2 * math.pi, device = device)) \
                      - 0.5 * d * torch.log(var_x(T-t)) - 0.5 * d * torch.log(var_obs) + 0.5 * d * torch.log(var_h(T-t)) \
                      + 0.5 * var_h(T-t) * torch.sum(torch.square(mean_x(T-t,x) / var_x(T-t) + y / var_obs), dim = 1, keepdim = True) \
                      - 0.5 * torch.sum(torch.square(mean_x(T-t,x)), dim = 1, keepdim = True) / var_x(T-t) \
                      - 0.5 * torch.sum(torch.square(y)) / var_obs

# true V0 network
V0_net_true = lambda x,y: -log_h(0.0, x, y.reshape(1,p)).squeeze()

# true Z network 
Z_net_true = lambda t,x,y: - var_h(T-t) * (mean_x(T-t,x) / var_x(T-t) + y / var_obs) * torch.exp(- beta * (T-t)) / var_x(T-t) \
                           + mean_x(T-t,x) * torch.exp(- beta * (T-t)) / var_x(T-t)

In [ ]:
# time-discretization settings
M = 50 # number of time steps

# V0 and Z neural network configuration
V0_net_config = {'layers': [16], 'standardization': standardization}
Z_net_config = {'layers': [d+16], 'standardization': standardization}
net_config = {'V0': V0_net_config, 'Z': Z_net_config}

# learning type
# learning_type = 'standard'
learning_type = 'iterative'                

# create model instance
model = cdt.core.model(state, obs, M, net_config, device = 'cpu')

In [ ]:
# optimization configuration (standard training)
I = 2000
optim_config = {'minibatch': 100, 
                'num_obs_per_batch': 10, 
                'num_iterations': I,
                'learning_rate' : 0.01, 
                'initial_required' : True}
# training
time_start = time.time() 
if learning_type == 'standard':
    model.train_standard(optim_config)
if learning_type == 'iterative':
    model.train_iterative(optim_config)
time_end = time.time()
time_elapsed = time_end - time_start
print("Training time (secs): " + str(time_elapsed))

In [ ]:
# plot loss over optimization iterations
plt.figure()
plt.plot(torch.arange(I), model.loss.to('cpu'), 'k.')
plt.xlabel('iteration', fontsize = 15)
plt.ylabel('loss', fontsize = 15)
plt.show()

In [ ]:
# auxiliary particle filter with true networks
def run_true_APF(initial_states, observations, num_samples):
    """
    Run auxiliary particle filter with true networks.
    
    Parameters
    ----------
    initial_states : initial states of X process (N, d)
    
    observations : sequence of observations to be filtered (K, p)

    num_samples : sample size (int)

    Returns
    -------
    dict containing:    
        states : X process at observation times (N, K+1, d) or (N, K*M+1, d) if full_path == True
        ess : effective sample sizes at unit times (K+1)        
        log_norm_const : log-normalizing constant estimates (K+1)
    """
    
    # initialize and preallocate
    N = num_samples
    Y = observations
    K = observations.shape[0]        
    X = initial_states        
    states = torch.zeros(N, K+1, d, device = device)
    states[:, 0, :] = X
    ess = torch.zeros(K+1, device = device)
    ess[0] = N
    log_norm_const = torch.zeros(K+1, device = device)
    log_ratio_norm_const = torch.tensor(0.0, device = device)
    
    # each observation
    for k in range(K):        

        # evaluate initial values V0
        V0 = V0_net_true(X, Y[k,:]) # size (N)            
        V = V0.clone()
        
        # each time interval
        for m in range(M):
            # time step 
            stepsize = model.stepsizes[m]
            t = model.time[m]

            # Brownian increment
            W = torch.sqrt(stepsize) * torch.randn(N, d, device = device) # size (N, d)
            
            # simulate V process forwards in time            
            Z = Z_net_true(t, X, Y[k,:]) # size (N, d)
            control = - Z.clone()
            drift_V = - 0.5 * torch.sum(torch.square(Z), 1) # size (N)                
            euler_V = V + stepsize * drift_V # size (N)
            V = euler_V + torch.sum(Z * W, 1) # size (N)

            # simulate X process forwards in time
            drift_X = b(X) + sigma * control
            euler_X = X + stepsize * drift_X
            X = euler_X + sigma * W

        # compute and normalize weights, compute ESS and normalizing constant
        log_weights = V + obs_log_density(X, Y[k,:]) - V0
        max_log_weights = torch.max(log_weights)
        weights = torch.exp(log_weights - max_log_weights)
        normalized_weights = weights / torch.sum(weights)
        ess[k+1] = 1.0 / torch.sum(normalized_weights**2)
        log_ratio_norm_const = log_ratio_norm_const + torch.log(torch.mean(weights)) + max_log_weights
        log_norm_const[k+1] = log_ratio_norm_const

        # resampling            
        ancestors = resampling(normalized_weights, N)
        X = X[ancestors,:]

        # store states 
        states[:, k+1, :] = X

    # output
    output = {'states' : states, 'ess' : ess, 'log_norm_const' : log_norm_const}

    return output


In [ ]:
# fully adapted auxiliary particle filter without time-discretization
def run_FA_APF(initial_states, observations, num_samples):
    """
    Run fully adapted auxiliary particle filter.
    
    Parameters
    ----------
    initial_states : initial states of X process (N, d)
    
    observations : sequence of observations to be filtered (K, p)

    num_samples : sample size (int)

    Returns
    -------
    dict containing:    
        states : X process at observation times (N, K+1, d) or (N, K*M+1, d) if full_path == True
        ess : effective sample sizes at unit times (K+1)        
        log_norm_const : log-normalizing constant estimates (K+1)
    """
    
    # initialize and preallocate
    N = num_samples
    Y = observations
    K = observations.shape[0]        
    X = initial_states        
    states = torch.zeros(N, K+1, d, device = device)
    states[:, 0, :] = X
    ess = torch.zeros(K+1, device = device)
    ess[0] = N
    log_norm_const = torch.zeros(K+1, device = device)
    log_ratio_norm_const = torch.tensor(0.0, device = device)
    
    # each observation
    for k in range(K): 
        # compute and normalize weights, compute ESS and normalizing constant
        log_weights = log_h(0.0, X, Y[k,:].reshape(1,p)).squeeze()
        max_log_weights = torch.max(log_weights)
        weights = torch.exp(log_weights - max_log_weights)
        normalized_weights = weights / torch.sum(weights)
        ess[k+1] = 1.0 / torch.sum(normalized_weights**2)
        log_ratio_norm_const = log_ratio_norm_const + torch.log(torch.mean(weights)) + max_log_weights
        log_norm_const[k+1] = log_ratio_norm_const

        # resampling            
        ancestors = resampling(normalized_weights, N)
        X = X[ancestors,:]

        # move using locally optimal proposal transition
        X = mean_h(X, Y[k,:].reshape(1,p)) + torch.sqrt(var_h(T)) * torch.randn(N, d, device = device)

        # store states 
        states[:, k+1, :] = X

    # output
    output = {'states' : states, 'ess' : ess, 'log_norm_const' : log_norm_const}

    return output

In [ ]:
# repeat particle filters
multiplier_level = list(range(1,11)) # controls level of misspecification
num_multiplier = len(multiplier_level)
K = 100 # number of observations
R = 100 # number of repeats
N = 2**6 # number of particles
BPF = {'ess' : torch.zeros(num_multiplier, R), 'log_estimate' : torch.zeros(num_multiplier, R)}
APF = {'ess' : torch.zeros(num_multiplier, R), 'log_estimate' : torch.zeros(num_multiplier, R)}
true_APF = {'ess' : torch.zeros(num_multiplier, R), 'log_estimate' : torch.zeros(num_multiplier, R)}
FA_APF = {'ess' : torch.zeros(num_multiplier, R), 'log_estimate' : torch.zeros(num_multiplier, R)}

for i in range(num_multiplier):
    # level of misspecification
    multiplier = float(multiplier_level[i])

    # simulate latent process and observations
    X0 = initial(1)
    X = torch.zeros(K+1, d)
    X[0,:] = X0.clone()
    Y = torch.zeros(K, p)
    for k in range(K):
        X[k+1,:] = model.simulate_diffusion(X[k,:])
        Y[k,:] = X[k+1,:] + multiplier * torch.sqrt(var_obs) * torch.randn(1,p)

    for r in range(R):
        # run particle filters
        BPF_output = model.run_BPF(X0.repeat((N,1)), Y, N)
        APF_output = model.run_APF(X0.repeat((N,1)), Y, N)
        true_APF_output = run_true_APF(X0.repeat((N,1)), Y, N)
        FA_APF_output = run_FA_APF(X0.repeat((N,1)), Y, N)

        # save average ESS%
        BPF_ESS = torch.mean(BPF_output['ess'] * 100 / N)
        APF_ESS = torch.mean(APF_output['ess'] * 100 / N)
        true_APF_ESS = torch.mean(true_APF_output['ess'] * 100 / N)
        FA_APF_ESS = torch.mean(FA_APF_output['ess'] * 100 / N)
        BPF['ess'][i,r] = BPF_ESS
        APF['ess'][i,r] = APF_ESS
        true_APF['ess'][i,r] = true_APF_ESS
        FA_APF['ess'][i,r] = FA_APF_ESS

        # save log-likelihood estimates
        BPF_log_estimate = BPF_output['log_norm_const'][-1]
        APF_log_estimate = APF_output['log_norm_const'][-1]
        true_APF_log_estimate = true_APF_output['log_norm_const'][-1]
        FA_APF_log_estimate = FA_APF_output['log_norm_const'][-1]
        BPF['log_estimate'][i,r] = BPF_log_estimate
        APF['log_estimate'][i,r] = APF_log_estimate
        true_APF['log_estimate'][i,r] = true_APF_log_estimate
        FA_APF['log_estimate'][i,r] = FA_APF_log_estimate

        # print output
        print('Multipler: ' + str(multiplier) + ' Repeat: ' + str(r)) 
        print('BPF ESS%: ' + str(BPF_ESS))
        print('APF ESS%: ' + str(APF_ESS)) 
        print('True-APF ESS%: ' + str(true_APF_ESS)) 
        print('FA-APF ESS%: ' + str(APF_ESS)) 
        print('BPF log-estimate: ' + str(BPF_log_estimate))
        print('APF log-estimate: ' + str(APF_log_estimate))
        print('True-APF log-estimate: ' + str(true_APF_log_estimate))
        print('FA-APF log-estimate: ' + str(FA_APF_log_estimate))

# save results
results = {'BPF' : BPF, 'APF' : APF, 'True-APF' : true_APF , 'FA-APF' : FA_APF}
torch.save(results, 'OU_robust.pt')